Attempting to create a simple program to handle a simple case of 2 body System

In [13]:
import numpy as np
import math
import random as rd
import pandas as pd 

List is created with following parameters : [Mass, X position, Y position, X velocity, Y velocity, X acceleration, Y acceleration]
This will be modified constantly to run values. For each time value, Then recalculate g force between

In [26]:

#Gravitational Constant
def G() :
    return 6.6743*(10**-11)

# Mass of Earth in kg
def EarthMass() :
    return 6*(10**24)

# Earth Orbital Velocity in m/s
def EarthOrbitalVel() :
    return 29722.2222

# Mass of Sun in kg
def SolarMass() :
    return 330000 * 6 * (10 ** 24)

#Astronomical Units
def AU():
    return 149597871 * 1000



# initializes dictionary for starting particles
def initalparticle(dictionary : dict, id : int, mass : float, x_pos : float, y_pos : float, x_vel : float, y_vel : float):
    dictionary.update({id : [mass, x_pos, y_pos, x_vel, y_vel, 0, 0]})

# Finds vector for calulations - returns vector pointing from particle 1 to particle 2 - allows direct calculation for gforce components
def findvector(dictionary, particle_1, particle_2) :
    x_1 = dictionary.get(particle_1)[1]
    x_2 = dictionary.get(particle_2)[1]
    y_1 = dictionary.get(particle_1)[2]
    y_2 = dictionary.get(particle_2)[2]
    
    dx = x_2 - x_1
    dy = y_2 - y_1
    magnitude = (dx**2 + dy**2)**.5
    sine = math.asin(dy/magnitude)
    cosine = math.acos(dx/magnitude)
    if(sine >= 0) :
        theta = cosine
    elif(sine < 0) :
        theta = -cosine
    
    #print("vector for calculation", [magnitude, theta * 180/math.pi])
    return [magnitude, theta]

# Force of Gravity of particle_2 on particle_1
def gforce(dictionary : dict, particle_1 : int, particle_2 : int) :
    vector = findvector(dictionary, particle_1, particle_2)
    m_2 = dictionary.get(particle_2)[0]
    acceleration = G()*m_2/(vector[0])**2

    
    dictionary.get(particle_1)[5] += acceleration*math.cos(vector[1])
    dictionary.get(particle_1)[6] += acceleration*math.sin(vector[1])
    #print(acceleration*math.cos(vector[1]),acceleration*math.sin(vector[1]))

# Calculates all of the gforces for all particle interactions
def apply_gforce(dictionary : dict, number_of_particles : int) :
    for x in range(0,number_of_particles):
        for y in range (0,number_of_particles):
            if x != y:
                #print(x, "and", y)
                gforce(dictionary, x, y)


# Approximates new position values
def position(dictionary : dict, index : int , dt : float) :
    x_vel = dictionary.get(index)[3]
    y_vel = dictionary.get(index)[4]
    x_acc = dictionary.get(index)[5]
    y_acc = dictionary.get(index)[6]

    dictionary.get(index)[1] += .5 * x_acc * (dt ** 2) + x_vel * dt 
    dictionary.get(index)[2] += .5 * y_acc * (dt ** 2) + y_vel * dt 
    #print("X axis   ", .5 * x_acc * (dt ** 2),  x_vel * dt , "   Y axis  ", .5 * y_acc * (dt ** 2), y_vel * dt)

# Approximates new velocity values 
def velocity(dictionary : dict, index : int , dt : float) :
    x_acc = dictionary.get(index)[5]
    y_acc = dictionary.get(index)[6]

    #print("X_ axis  ", dictionary.get(index)[3], x_acc * dt, "  Y_axis  ", dictionary.get(index)[4], y_acc * dt)

    dictionary.get(index)[3] += x_acc * dt
    dictionary.get(index)[4] += y_acc * dt
    
    

# clears both x and y acceleration so algorithm continues to work
def reset_acc(dictionary : dict, index : int) :
    dictionary.get(index)[5] = 0
    dictionary.get(index)[6] = 0

# records x and y positions for later graphing - fills x_list and y_list with another list that contains respective positions for each particle
def record_pos(dictionary : dict, x_list : list, y_list : list) :
    temp_x = []
    temp_y = []
    
    for index in dictionary :
        temp_x.append(dictionary.get(index)[1])
        temp_y.append(dictionary.get(index)[2])
    
    x_list.append(temp_x)
    y_list.append(temp_y)




# moves forward time in simulation - order for position / velocity matters 
def model(dictionary : dict, dt : float, x_list : list, y_list : list) :
    for index in dictionary :
        position(dictionary, index, dt)
        velocity(dictionary, index, dt)
        reset_acc(dictionary, index)

# takes particle index and prints out cvs text input for desmos graphing (hopefully)
def csv(particle : int, x_list : int, y_list) :
    for index in range(0, len(x_list)):
        x_pos = x_list[index][particle]
        y_pos = y_list[index][particle]
        print (f"{x_pos}, {y_pos}")
    
def to_df(particleinfo, x_list : int, y_list) :
    total = pd.DataFrame(columns=["x","y"])
    for i in particleinfo:
        for index in range(0, len(x_list)):
            x_pos = x_list[index][i]
            y_pos = y_list[index][i]
            df = pd.DataFrame({'x': x_pos, 'y': y_pos}, index=[index])  # Specify the index when creating the DataFrame
            total = pd.concat([df, total], ignore_index=True)
    return total


Testing 5 body sun with 4 Earths at theta 0, pi/2, pi, 3pi/2

In [27]:
particleinfo = {}
x_list = []
y_list = []
number_of_particles = 2


# Initializes amount of particles  149597871 * 1000
initalparticle(particleinfo, 0, EarthMass(), AU(), 0, 0, EarthOrbitalVel())
#initalparticle(particleinfo, 1, EarthMass(), -AU(), 0, 0, -EarthOrbitalVel())
#initalparticle(particleinfo, 2, EarthMass(), 0, AU(), -EarthOrbitalVel(), 0)
#initalparticle(particleinfo, 3, EarthMass(), 0, -AU(), EarthOrbitalVel(), 0)
initalparticle(particleinfo, 1, SolarMass(), 0, 0, 0, 0)


#print("inital conditions      ", particleinfo)


# Runs program and records - in seconds
delta_t = 60 * 60
trials = 24 * 365 * 10

for index in range(trials):
    apply_gforce(particleinfo,number_of_particles)
    model(particleinfo, delta_t, x_list, y_list)
    if (index % (24 * 12) == 0) :
        record_pos(particleinfo, x_list, y_list)


#print("x_list         ", x_list)
#print("y_list         ", y_list)

# for i in particleinfo :
#     csv(i,x_list,y_list)
#     print("\n\n")


In [28]:
df = to_df(particleinfo,x_list,y_list)
df

,x,y
0,7.259103e+05,2.877498e+07
1,7.950019e+05,2.862201e+07
2,8.513652e+05,2.845685e+07
3,8.928918e+05,2.828219e+07
4,9.180227e+05,2.810104e+07
...,...,...
605,1.015436e+11,1.098721e+11
606,1.218674e+11,8.677224e+10
607,1.370378e+11,6.000580e+10
608,1.464133e+11,3.070440e+10


In [ ]:
pd.DataFrame(particleinfo)

,0,1
0,6000000000000000000000000,1980000000000000000000000000000
1,-85889292349.354507,713597.464695
2,-130349517757.945236,28798634.881061
3,24270.895779,-0.073548
4,-16025.405903,0.138629
5,0,0
6,0,0


In [16]:
pd.DataFrame(x_list)

,0,1
0,1.495978e+11,0.115953
1,1.464133e+11,9650.289093
2,1.370378e+11,38060.954760
3,1.218674e+11,84031.701477
4,1.015436e+11,145619.042714
...,...,...
300,-1.533496e+11,918022.720324
301,-1.450564e+11,892891.795354
302,-1.313527e+11,851365.238633
303,-1.127528e+11,795001.906036


In [8]:
df_particle

,Particle_ID,X,Y
0,1,1.464133e+11,3.070440e+10
1,1,9.650289e+03,6.624257e+02
